In [33]:
import sys
sys.path.append("..")
import mindspore
from mindspore import context
from mindspore import nn
from mindspore import Tensor
from mindspore import ops
from src.bert import BertForPretraining
from src.config import BertConfig
from src.tokenizer import BertTokenizer
from model import BertBinaryClassificationModel
from mindnlp.mindnlp.mindtext.dataset.classification.sst import SST2Dataset


In [34]:
context.set_context(mode=context.GRAPH_MODE)
#context.set_context(mode=context.PYNATIVE_MODE)
dataset = SST2Dataset(paths='SST-2',
                      tokenizer="bert-base-uncased",
                      max_length=128,
                      truncation_strategy=True,
                      columns_list=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
                      test_columns_list=['input_ids', 'token_type_ids', 'attention_mask'],
                      batch_size=16)

sst_2_ds = dataset()

SST-2 dataset loadding: 100%|██████████| 67349/67349 [00:04<00:00, 15668.09it/s] 
SST-2 train dataset sentence preprocess bar(tokenize).: 100%|██████████| 67349/67349 [00:22<00:00, 3052.18it/s]
SST-2 train dataset processing.: 100%|██████████| 67349/67349 [00:03<00:00, 17087.49it/s] 
[WARNING] ME(17740:140613899089664,MainProcess):2022-11-11-15:25:17.453.595 [mindspore/dataset/core/validator_helpers.py:806] 'TypeCast' from mindspore.dataset.transforms.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'TypeCast' from mindspore.dataset.transforms instead.
Writing data to .mindrecord file: 100%|██████████| 67349/67349 [02:50<00:00, 394.19it/s]
SST-2 dev dataset sentence preprocess bar(tokenize).: 100%|██████████| 872/872 [00:00<00:00, 1909.87it/s]
SST-2 dev dataset processing.: 100%|██████████| 872/872 [00:00<00:00, 16450.17it/s]
[WARNING] ME(17740:140613899089664,MainProcess):2022-11-11-15:28:09.600.508 [mindspore/dataset/core/validator_helpers.py:8

In [35]:
print(sst_2_ds.keys())
# dev_dataset batch_size 8
# dev_dataset = sst_2_ds1['dev']
# train_dataset batch_size 32
train_dataset = sst_2_ds['train']
test_dataset = sst_2_ds['test']
dev_dataset = sst_2_ds['dev']
type(test_dataset)
print(train_dataset.get_col_names())

print(train_dataset.get_batch_size(), train_dataset.get_dataset_size())
print(dev_dataset.get_batch_size(), dev_dataset.get_dataset_size())

dict_keys(['train', 'dev', 'test'])
['input_ids', 'token_type_ids', 'attention_mask', 'label']
16 4210
16 55


In [36]:
# batch and tokenizer -> (input_ids ,token_type_ids, attention_mask, label)
print(train_dataset.get_col_names())
for data in train_dataset.create_dict_iterator():
    print(data['input_ids'].shape)
    print(data['attention_mask'].shape)
    print(data['token_type_ids'].shape)
    print(data['label'].shape)
    print(len(data))
    break
for input_ids, token_type_ids, attention_mask, label in train_dataset.create_tuple_iterator():
    print(input_ids, token_type_ids, attention_mask, label)
    break

['input_ids', 'token_type_ids', 'attention_mask', 'label']
(16, 128)
(16, 128)
(16, 128)
(16, 1)
4
[[  101  4760  2149 ...     0     0     0]
 [  101  6256   102 ...     0     0     0]
 [  101 11867  6340 ...     0     0     0]
 ...
 [  101  5636 24372 ...     0     0     0]
 [  101  2013  1996 ...     0     0     0]
 [  101  2003  1037 ...     0     0     0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]] [[1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]]


In [37]:
# test load ckpt
from src.config import BertConfig
ckpt_file = "checkpoint/bert_L4_H128_step_308749_card_id_0.ckpt"
config_file = 'config/bert_L4_H128_config.json'
config = BertConfig(config_file)
print(config.vocab_size)
model = BertForPretraining(config)

before_model = BertForPretraining(config)
dict_before = before_model.bert.parameters_dict()

dict_after = mindspore.load_checkpoint(ckpt_file)
mindspore.load_param_into_net(model, dict_after)
model = model.bert
dict_after = model.parameters_dict()
import numpy as np
list_before = list(dict_before.values())
list_after = list(dict_after.values())

print(np.allclose(list_before[2].asnumpy(), list_after[2].asnumpy(), rtol=1e-3))

30522
False


In [38]:
bert_sst_2 = BertBinaryClassificationModel(config, ckpt_file)
# test output 

for input_ids, token_type_ids, attention_mask, label in train_dataset.create_tuple_iterator():
    print(len(input_ids))
    loss, logits = bert_sst_2(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, label=label)
    break

for item in bert_sst_2.get_parameters():
    print(item)

16
Parameter (name=classifier.weight, shape=(2, 128), dtype=Float32, requires_grad=True)
Parameter (name=classifier.bias, shape=(2,), dtype=Float32, requires_grad=True)
Parameter (name=bert.embeddings.word_embeddings.embedding_table, shape=(30522, 128), dtype=Float32, requires_grad=True)
Parameter (name=bert.embeddings.position_embeddings.embedding_table, shape=(512, 128), dtype=Float32, requires_grad=True)
Parameter (name=bert.embeddings.token_type_embeddings.embedding_table, shape=(2, 128), dtype=Float32, requires_grad=True)
Parameter (name=bert.embeddings.layer_norm.gamma, shape=(128,), dtype=Float32, requires_grad=True)
Parameter (name=bert.embeddings.layer_norm.beta, shape=(128,), dtype=Float32, requires_grad=True)
Parameter (name=bert.encoder.layer.0.attention.self_attn.query.weight, shape=(128, 128), dtype=Float32, requires_grad=True)
Parameter (name=bert.encoder.layer.0.attention.self_attn.query.bias, shape=(128,), dtype=Float32, requires_grad=True)
Parameter (name=bert.encoder

In [39]:
import os
def getpwd():
    pwd = sys.path[0]
    if os.path.isfile(pwd):
        pwd = os.path.dirname(pwd)
    return pwd

In [40]:
def train_loop(model, dataset, optimizer):
    def forward_fn(input_ids, attention_mask, token_type_ids, label):
        loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        label=label)
        return loss, logits
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    def train_step(input_ids, attention_mask, token_type_ids, label):
        (loss, _), grads = grad_fn(input_ids, attention_mask, token_type_ids, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()

    for batch, (input_ids, attention_mask, token_type_ids, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(input_ids, attention_mask, token_type_ids, label)
        if batch % 10 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [41]:
def test_loop(model, dataset, output_file, current_step, current_epoch):
#def test_loop(model, dataset):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for input_ids, attention_mask, token_type_ids, label in dataset.create_tuple_iterator():
        loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        label=label)
        total += len(input_ids)
        test_loss += loss.asnumpy()
        correct_loop = (logits.argmax(1) == label.view(-1)).asnumpy().sum()
        correct += correct_loop
    test_loss /= num_batches
    correct /= total
    if correct*100 >= 82.84:
        output_ckpt_file = os.path.join(output_file, "{epoch}_epoch_{step}_step_acc_{acc}%.ckpt".format(
            epoch=current_epoch, step=current_step, acc=(correct*100)
        ))
        mindspore.save_checkpoint(model, output_ckpt_file)
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [42]:
def train_and_test(model, train_dataset, dev_dataset, optimizer, current_epoch):
    output_file = os.path.join(getpwd(), "outputs")
    if not os.path.exists(output_file):
        try:
            os.mkdir(output_file)
        except FileExistsError:
            pass
    def forward_fn(input_ids, attention_mask, token_type_ids, label):
        loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        label=label)
        return loss, logits
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    def train_step(input_ids, attention_mask, token_type_ids, label):
        (loss, _), grads = grad_fn(input_ids, attention_mask, token_type_ids, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = train_dataset.get_dataset_size()
    model.set_train(True)

    for batch, (input_ids, attention_mask, token_type_ids, label) in enumerate(train_dataset.create_tuple_iterator()):
        loss = train_step(input_ids, attention_mask, token_type_ids, label)
        if batch % 10 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
        # add test
        if batch % 100 == 0:
            print("In epoch {} _ batch {} testing".format(current_epoch + 1, current))
            test_loop(model, dev_dataset, output_file, batch, current_epoch)
            # test_loop(model, dev_dataset)

In [43]:
# train set
epoch_num = 10
config_file = 'config/bert_L4_H128_config.json'
ckpt_file = "checkpoint/bert_L4_H128_step_308749_card_id_0.ckpt"
config = BertConfig(config_file)
model = BertForPretraining(config)
bert_sst_2 = BertBinaryClassificationModel(config, ckpt_file)

# freeze_layer for classify task
# freeze_params = list(filter(lambda x: 'bert.encoder.layer.0' in x.name \
#                                     or 'bert.encoder.layer.1' in x.name \
#                                     or 'bert.encoder.layer.2' in x.name \
#                                     or 'bert.embeddings.token_type_embeddings' in x.name \
#                                     or 'bert.embeddings.position_embeddings' in x.name \
#                                     or 'bert.embeddings.word_embeddings' in x.name, \
#                             bert_sst_2.bert.trainable_params()))
# freeze_params = list(filter(lambda x: 'bert' in x.name, bert_sst_2.bert.trainable_params()))
# for item in freeze_params:
#     item.requires_grad = False

In [44]:
params = bert_sst_2.trainable_params()
# print(params)

In [45]:
# AdamWeightDecay
from src.optimization import BertLearningRate
# lr_schedule_for_backbone = BertLearningRate(learning_rate=5e-5,
#                                    end_learning_rate=0.0,
#                                    warmup_steps=int(train_dataset.get_dataset_size() * epoch_num * 0.1),
#                                    decay_steps=train_dataset.get_dataset_size() * epoch_num,
#                                    power=1.0)

lr_schedule = BertLearningRate(learning_rate=5e-5,
                                   end_learning_rate=0.0,
                                   warmup_steps=int(train_dataset.get_dataset_size() * epoch_num * 0.1),
                                   decay_steps=train_dataset.get_dataset_size() * epoch_num,
                                   power=1.0)
# decay_params = list(filter(lambda x: 'gamma' not in x.name \
#                                         and 'beta' not in x.name \
#                                         and 'bias' not in x.name, params))
# other_params = list(filter(lambda x: 'gamma' in x.name \
#                                         or 'beta' in x.name \
#                                         or 'bias' in x.name, params))
# group_params = [{'params': decay_params, 'weight_decay': 0.01},
#                     {'params': other_params},
#                     {'order_params': params}]

# classify_params = list(filter(lambda x: 'classifier' in x.name, \
#                             bert_sst_2.trainable_params()))

# other_params = list(filter(lambda x: 'classifier' not in x.name, \
#                             bert_sst_2.trainable_params()))

# group_params = [{'params': classify_params, 'weight_decay': 0.01, 'lr': lr_schedule_for_finetuing},
#                 {'params': other_params, 'lr': lr_schedule_for_finetuing}]

optimizer = nn.AdamWeightDecay(params, lr_schedule, eps=1e-8)


In [46]:
for epoch in range(epoch_num):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_and_test(bert_sst_2, train_dataset, dev_dataset, optimizer, epoch)
print("Done!")

Epoch 1
-------------------------------
loss: 0.692476  [  0/4210]
In epoch 1 _ batch 0 testing
Test: 
 Accuracy: 51.6%, Avg loss: 0.691844 

loss: 0.680482  [ 10/4210]
loss: 0.664221  [ 20/4210]
loss: 0.707480  [ 30/4210]
loss: 0.693433  [ 40/4210]
loss: 0.674515  [ 50/4210]
loss: 0.681755  [ 60/4210]
loss: 0.676224  [ 70/4210]
loss: 0.681729  [ 80/4210]
loss: 0.711949  [ 90/4210]
loss: 0.687227  [100/4210]
In epoch 1 _ batch 100 testing
Test: 
 Accuracy: 50.9%, Avg loss: 0.692012 

loss: 0.675782  [110/4210]
loss: 0.683089  [120/4210]
loss: 0.688611  [130/4210]
loss: 0.693463  [140/4210]
loss: 0.673625  [150/4210]
loss: 0.682184  [160/4210]
loss: 0.689173  [170/4210]
loss: 0.687371  [180/4210]
loss: 0.671180  [190/4210]
loss: 0.715777  [200/4210]
In epoch 1 _ batch 200 testing
Test: 
 Accuracy: 50.9%, Avg loss: 0.693102 

loss: 0.697979  [210/4210]
loss: 0.695149  [220/4210]
loss: 0.668432  [230/4210]
loss: 0.675704  [240/4210]
loss: 0.644326  [250/4210]
loss: 0.701753  [260/4210]
lo